In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
import pandas as pd
import emcee
from scipy.optimize import minimize
from astropy.io import ascii
import corner
from multiprocessing import Pool
import os
from IPython.display import display, Math

# Constants in CGS units
c = 3e10            # Speed of light [cm/s]
h = 6.626e-27       # Planck constant [erg·s]
k_B = 1.38e-16      # Boltzmann constant [erg/K]
sigma = 5.67e-5     # Stefan-Boltzmann constant [erg/cm^2/s/K^4]
G = 6.674e-8        # Gravitational constant [cm^3/g/s^2]
M_sun = 1.989e33    # Solar mass [g]
m_p = 1.6726e-24    # Proton mass [g]
L_Edd_factor = 1.26e38  # Eddington luminosity factor

# Cosmological parameters
H0 = 2.195e-18      # Hubble constant [s^-1]
Omega_m = 0.308     # Matter density parameter
Omega_Lambda = 0.692  # Dark energy density parameter


# Frequency range [Hz]
frequencies = np.logspace(10, 18, 100)


# Redshift of the source
z = float(input("Enter the redshift (z) of the AGN: "))


#viewing angle(deg to rad)
theta_deg = float(input("Enter the viewing angle (degrees): "))
theta=np.deg2rad(theta_deg)


def luminosity_distance(z):
    """
    Compute the luminosity distance for a given redshift z.
    Returns:
    - d_L: Luminosity distance [cm]
    """
    def integrand(z_prime):
        return 1.0 / np.sqrt(Omega_m * (1 + z_prime)**3 + Omega_Lambda)
    integral, _ = quad(integrand, 0, z)
    d_L = c * (1 + z) * integral / H0  # [cm]
    return d_L

def disk_temperature(r, M_bh, M_dot):
    """
    Temperature profile of the accretion disk as a function of radius.
    """
    R_s = 2 * G * M_bh / c**2  # Schwarzschild radius [cm]
    R_in = 3 * R_s
    if r < R_in:
        return 0.0
    factor = 3 * M_dot * R_s * c**2 / (16 * np.pi * sigma * r**3)
    T = (factor * (1 - np.sqrt(R_in / r)))**0.25
    return np.clip(T, 1e1, 1e12)

def planck_function(nu, T):
    """
    Planck blackbody function at frequency nu and temperature T.
    """
    if T == 0.0:
        return 0.0
    exponent = h * nu / (k_B * T)
    if exponent > 700:
        return 0.0
    return (2 * h * nu**3 / c**2) / (np.exp(exponent) - 1)

def flux_density(nu, M_bh, M_dot, theta, z):
    """
    Calculate the disk flux density at frequency nu.
    """
    R_s = 2 * G * M_bh / c**2  # Gravitational radius
    R_in = 3 * R_s             # Inner disk radius
    R_out = 1000 * R_s         # Outer disk radius

    def integrand(r):
        return 2 * np.pi * r * np.cos(theta) * planck_function(nu, disk_temperature(r, M_bh, M_dot))

    # Use logarithmically spaced radii for integration
    r_vals = np.logspace(np.log10(R_in), np.log10(R_out), 100)
    integrand_vals = [integrand(r) for r in r_vals]
    # Integrate in log-space: d(log r) = dr/r
    integral = np.trapz(np.array(integrand_vals)*r_vals, np.log(r_vals))
    d_L = luminosity_distance(z)
    F_nu = integral / d_L**2
    return F_nu

def simulate_spectrum(frequencies, M_bh, M_dot, theta, z):
    """
    Simulate the accretion disk spectrum for given parameters.
    The simulation computes the model flux density at each (redshift-corrected) frequency.
    """
    # Here, frequencies are in the observer frame.
    return np.array([flux_density(nu, M_bh, M_dot, theta, z) for nu in frequencies*(1+z)])


def log_likelihood(params, frequencies, fluxes, flux_errors, z):

    M_bh, M_dot, log_f = params

    M_bh_linear = 10**M_bh
    M_dot_linear = 10**M_dot

    # Compute the model spectrum (flux density F_nu) in the quasar rest frame.

    model_fluxes = simulate_spectrum(frequencies, M_bh_linear, M_dot_linear, theta, z) * frequencies *(1+z)

    log_model_fluxes = np.log10(model_fluxes)
    sigma2 = flux_errors**2 + np.exp(2 * log_f)
    return -0.5 * np.sum((fluxes - log_model_fluxes)**2 / sigma2 + np.log(sigma2))

def log_prior(params):
   M_bh, M_dot,log_f= params
   M_bh_linear = 10**M_bh
   M_dot_linear = 10**M_dot
   Mdot_Edd = L_Edd_factor * (M_bh_linear / M_sun) / (0.1 * c**2)
    # Prior
   if (log_M_min + np.log10(M_sun) < M_bh < log_M_max + np.log10(M_sun)) and  (log_Mdot_min < M_dot < np.log10(Mdot_Edd))  and (log_f_min < log_f <lof_f_max) :

    return 0.0

   return -np.inf


def log_posterior(params, frequencies, fluxes, flux_errors, z):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params, frequencies, fluxes, flux_errors, z)

# ------------------------------
# Load initial CSV data and convert to log scale
# ------------------------------
csv_file = "data.csv"
csv_data = pd.read_csv(csv_file)
frequencies_csv_log = np.log10(csv_data.iloc[:, 0].values)  # Frequency [Hz]
fluxes_csv = csv_data.iloc[:, 1].values
fluxes_csv_log = np.log10(fluxes_csv)
flux_errors = 0.1 * fluxes_csv
flux_error_log_upper = np.log10(fluxes_csv + flux_errors) - np.log10(fluxes_csv)
flux_errors_log = flux_error_log_upper  # Using upper errors



# Prior bounds (min, max) for each parameter

log_M_min = (float(input("Min log(M): ")))
log_M_max = (float(input("Max log(M): ")))

log_Mdot_min = (float(input("Min log(Mdot): ")))

log_f_min = (float(input("Min log_f): ")))
lof_f_max = (float(input("Max log_f): ")))


# Initial guess for the parameters (in log10)

log_M_init = float(input("Initial guess for M: "))
log_Mdot_init = float(input("Initial guess for log(Mdot [M_sun/year]): "))
log_f_init = float(input("Initial guess for log_f: "))


initial = np.array([np.log10(log_M_init * M_sun),
                    np.log10(log_Mdot_init * M_sun/(365 * 24 * 3600)),log_f_init])


# MCMC setup
ndim = 3
nwalkers = int(input("Number of walkers: "))
nsteps = int(input("Number of steps: "))
nburn = int(input("Number of steps to discard: "))



pos = initial + np.abs(2e-1 * np.random.randn(nwalkers, ndim))  # Initial walker positions

# Define output directory and HDF5 filename
out_dir = "./"  # or any directory you want
filename = os.path.join(out_dir, "output.h5")


# Set up the backend and reset
backend = emcee.backends.HDFBackend(filename)
backend.reset(nwalkers, ndim)

with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior,
                                    args=(10**frequencies_csv_log, fluxes_csv_log, flux_errors_log, z),
                                    pool=pool,
                                    backend=backend)
    sampler.run_mcmc(pos, nsteps, progress=True)

# Analyze the MCMC chains
samples = sampler.get_chain(discard=nburn, thin=15, flat=True)

# Convert MCMC samples back to physical units for plotting
M_bh_linear_samples = 10**samples[:, 0] / M_sun
M_dot_linear_samples = 10**samples[:, 1] * (365*24*3600) / M_sun
log_f_samples = samples[:, 2]

M_bh_ml_linear = 10**initial[0] / M_sun
M_dot_ml_linear = 10**initial[1] * (365*24*3600) / M_sun
log_f_ml = initial[2]

# Create the corner plot
fig = corner.corner(np.vstack([M_bh_linear_samples, M_dot_linear_samples,log_f_samples]).T,
                    labels=["$M_{\\rm bh}$ ($M_\\odot$)", "$\\dot{M}$ ($M_\\odot$/yr)","log$_{10}(f)$"],
                    truths=[M_bh_ml_linear, M_dot_ml_linear,log_f_ml])
plt.show()

fig.savefig("corner_plot.png")



overlay_size = int(input("Number of curves to be shown in the overlay: "))

flat_samples = samples  # raw chain in log-space (for the first two parameters) and theta as is
scaled_samples = 10**samples.copy()  # Exponentiate first two columns; theta remains unchanged in the copy

# Convert first column: black hole mass from grams to solar masses
scaled_samples[:, 0] /= M_sun
# Convert second column: accretion rate from g/s to M_sun/yr
scaled_samples[:, 1] *= 365 * 24 * 3600 / M_sun


# Select a random subset of indices for plotting
inds = np.random.randint(len(flat_samples), size=overlay_size)

for ind in inds:
    sample = scaled_samples[ind]
    M_bh = sample[0]       # in M_sun
    M_dot = sample[1]      # in M_sun/yr


    # Convert back to cgs units for simulation:
    M_bh_cgs = M_bh * M_sun
    M_dot_cgs = M_dot * M_sun / (365 * 24 * 3600)

    spectrum = simulate_spectrum(frequencies, M_bh_cgs, M_dot_cgs, theta, z)
    label = f"M_bh={M_bh:.0e} M$_{{\odot}}$, M$_{{\dot{{M}}}}$={M_dot:.1f} M$_{{\odot}}$/yr, $\\theta$={np.rad2deg(theta):.1f}°"

    # Convert model flux to observer (here multiplying by frequencies*(1+z))
    plt.loglog(frequencies, spectrum*frequencies*(1+z), linewidth=0.6, color='orange')


plt.loglog(10**frequencies_csv_log,10**fluxes_csv_log, 'ro', label="Data", markersize=5,color='blue')

# Plot settings
plt.xlabel("Frequency [Hz]")
plt.ylabel("Flux Density*Frequency [erg/s/cm²]")



#Optinal limits

#plt.ylim(1e-15, 1e-11)
#plt.xlim(1e13, 1e16)



plt.title("Theoretical Accretion Disk Spectra")
plt.legend()
plt.grid(which="both", linestyle="--", alpha=0.5)
plt.show()


# Create a copy of the samples to avoid modifying the original array.
flat_samples = samples.copy()

# Convert the first two columns from log-space to linear scale:
# For black hole mass and accretion rate.
flat_samples[:, 0:2] = 10**flat_samples[:, 0:2]

# Convert black hole mass from grams to solar masses,
# and accretion rate from g/s to M_sun/yr.
flat_samples[:, 0] /= M_sun
flat_samples[:, 1] *= (365 * 24 * 3600) / M_sun
# The theta column (index 2) remains unchanged (in radians).

# Define LaTeX-formatted labels for the parameters.
labels = ["M_{bh}", "\\dot{M}"]

ndim = flat_samples.shape[1]

# Loop through each parameter and compute the 16th, 50th, and 84th percentiles.
for i in range(ndim):
    # Compute percentiles: 16th, 50th (median), and 84th.
    percentiles = np.percentile(flat_samples[:, i], [16, 50, 84])
    median = percentiles[1]
    lower_err = median - percentiles[0]
    upper_err = percentiles[2] - median

    if i == 0:  # Black hole mass in solar masses.
        txt = r"\mathrm{{{3}}} = {0:.3e}_{{-{1:.3e}}}^{{{2:.3e}}}\, M_{{\odot}}"
        formatted_txt = txt.format(median, lower_err, upper_err, labels[i])
    elif i == 1:  # Accretion rate in solar masses per year.
        txt = r"\mathrm{{{3}}} = {0:.3e}_{{-{1:.3e}}}^{{{2:.3e}}}\, M_{{\odot}}/yr"
        formatted_txt = txt.format(median, lower_err, upper_err, labels[i])


    display(Math(formatted_txt))



